---
title: Planetary Motion
format:
  live-html:
    toc: true
    toc-location: right
pyodide:
  autorun: false
  packages:
    - matplotlib
    - numpy
    - scipy
---


```{pyodide}
#| edit: false
#| echo: false
#| execute: true

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Set default plotting parameters
plt.rcParams.update({
    'font.size': 12,
    'lines.linewidth': 1,
    'lines.markersize': 5,
    'axes.labelsize': 11,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'xtick.top': True,
    'xtick.direction': 'in',
    'ytick.right': True,
    'ytick.direction': 'in',
})

def get_size(w, h):
    return (w/2.54, h/2.54)
```
## Physical Model

The motion of planets around the Sun is a classic problem in physics that beautifully demonstrates Newton's laws of motion and universal gravitation. While this motion might seem very different from a spring pendulum, the mathematical description is surprisingly similar. The main difference is that instead of a spring force, we now have gravity as our central force.

In both cases, we describe the motion using two coordinates: the distance from the center ($r$) and an angle ($\theta$). For planetary motion, $r$ is the distance between the planet and the Sun, and $\theta$ describes the planet's angular position in its orbit.

The equations of motion contain two parts: The first equation describes the radial acceleration ($\ddot{r}$), and the second describes the angular acceleration ($\ddot{\theta}$):

\begin{eqnarray}
\ddot{r}&=&r\dot{\theta}^2-\frac{G\, M}{r^2}\\
\ddot{\theta}&=&-\frac{1}{r}2\dot{r}\dot{\theta}
\end{eqnarray}

In the first equation, the term $r\dot{\theta}^2$ represents the centrifugal effect - the tendency of the planet to move away from the Sun due to its orbital motion. The term $-\frac{G\, M}{r^2}$ is Newton's gravitational force (divided by mass), pulling the planet toward the Sun. $G$ is the gravitational constant, and $M$ is the mass of the Sun.

The second equation describes how the angular motion changes. The term $2\dot{r}\dot{\theta}$ represents the coupling between radial and angular motion - as the planet moves closer to or farther from the Sun, its angular velocity must change to conserve angular momentum, similar to how an ice skater spins faster when pulling their arms in.

The solution to these equations gives us the famous orbital equation:

$$
r(\theta)=\frac{p}{1+\epsilon \cos(\theta)}
$$

This is the equation of a conic section, where $p$ and $\epsilon$ determine the orbit's shape. The parameter $p$ is related to the angular momentum $L$:

\begin{equation}
p=\frac{L^2}{G M m^2}
\end{equation}

The eccentricity $\epsilon$ depends on both the energy $E$ and angular momentum $L$:

\begin{equation}
\epsilon=\sqrt{1+\frac{2\frac{E}{m}\frac{L^2}{m^2}}{G^2M^2}}
\end{equation}

When $0 < \epsilon < 1$, we get an elliptical orbit (the case for all planets in our solar system). A perfect circle occurs when $\epsilon = 0$. These equations, derived by Newton and studied by Kepler, explain not only planetary orbits but also the paths of comets, artificial satellites, and many other celestial objects.

```{pyodide}
#| autorun: false

def planetary_motion(state, time ):
    g0 = state[1]
    g1 = state[0]*state[3]**2 - G*M/(state[0]**2)
    g2 = state[3]
    g3 = -2.0*state[1]*state[3]/state[0]
    return np.array([g0, g1, g2, g3])
```

## Numerical Solution

For the numerical solution, we will use the `odeint` function from the `scipy.integrate` module. This function solves ordinary differential equations (ODEs) given an initial state and a time array. We will integrate the equations of motion for a planet with an initial radius $r_0$, radial velocity $v_0$, angle $\theta_0$, and angular velocity $\omega_0$.

### Initial Parameters: Planets

We first define the initial parameters for the planet's motion. We set the mass of the Sun $M=1$ and the mass of the planet $m=1$ for simplicity. The gravitational constant $G$ is set to $4\pi^2$ to simplify the equations. We also define the initial radius $r_0$, radial velocity $v_0$, angle $\theta_0$, and angular velocity $\omega_0$.

```{pyodide}
#| autorun: false

G=4*np.pi**2
M=1 # mass of the sun
m=1 # mass of the earth

N = 10000
state = np.zeros ([4])
r_o= 1.8 # initial radius
v_o = 0 # initial radial velocity
theta_o = 0 # initial angle
omega_o = 1# initial angular velocity, for 2.222 it becomes circular


state[0]=r_o
state[1]=v_o
state[2] = theta_o
state[3] = omega_o

time = np.linspace(0, 3, N)
```

### Solution: Planets

T

```{pyodide}
#| autorun: false

answer = odeint ( planetary_motion , state , time )

xdata = answer[:,0]*np.cos(answer[:,2])
ydata = answer[:,0]*np.sin(answer[:,2])
```

```{pyodide}
#| autorun: false

# ellipse parameters

L=m*r_o**2*omega_o # angular momentum
E=0.5*m*(v_o**2+r_o**2*omega_o**2)-G*M*m/r_o

p=(L/m)**2/(G*M)
e=np.sqrt(1+(2*E*L**2/(m**3))/(G*G*M*M))
```

### Plotting: Planets

We can now plot the numerical solution of the planetary motion. The black dashed line represents the analytical solution for the orbit of the planet. The black solid line shows the numerical solution obtained by integrating the equations of motion.

#### Trajectory

```{pyodide}
#| autorun: false

# analytical solution
theta=np.linspace(0,2*np.pi,1000)
r=p/(1+e*np.cos(theta))


fig=plt.figure(1, figsize = get_size(7,7))
plt.plot(xdata,ydata,'k-',lw=5,alpha=0.3,label='numerical')
plt.plot(-r*np.cos(theta),r*np.sin(theta),'k--',lw=1,label='analytical')
plt.plot(0,0,'o')
plt.xlim(-2,2)
plt.ylim(-2,2)
plt.legend()
plt.tight_layout()
plt.show()
```

#### Energy

Finally, we can plot the total energy of the planet as a function of time. The total energy is the sum of the kinetic and potential energy of the planet. We can see that the energy is conserved over time, as expected for a system with no external forces.

```{pyodide}
#| autorun: false

Etot=0.5*m*(answer[:,1]**2+answer[:,0]**2*answer[:,3]**2)-G*M*m/answer[:,0]
Ekin=0.5*m*(answer[:,1]**2+answer[:,0]**2*answer[:,3]**2)
Epot=-G*M*m/answer[:,0]
```

```{pyodide}
#| autorun: false

plt.figure(figsize=get_size(7, 6))
plt.plot(time,Ekin,label="E_kin")
plt.plot(time,Epot,label="E_pot")
plt.plot(time,Etot)
plt.xlabel('time')
plt.ylabel('energy')
plt.ylim(-100,100)
plt.legend()
plt.tight_layout()
plt.show()
```